In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re as re
import matplotlib.pyplot as plt
import spacy

import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
stops = stopwords.words('english')

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
#import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from sklearn.decomposition import LatentDirichletAllocation as LDA
from gensim import corpora

from collections import Counter

from spacy.attrs import ORTH, NORM





import re
import contractions
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from autocorrect import Speller






In [ ]:
# If there are new keywords encountered that are a mix of letters and numbers, it would be beneficial to create special tokenizer classes for it

nlp = spacy.load('en_core_web_lg')
nlp.tokenizer.add_special_case('5g', [{ORTH: '5g', NORM: '5g'}])
nlp.tokenizer.add_special_case('2g', [{ORTH: '2g', NORM: '2g'}])
nlp.tokenizer.add_special_case('3g', [{ORTH: '3g', NORM: '3g'}])
nlp.tokenizer.add_special_case('4g', [{ORTH: '4g', NORM: '4g'}])

In [ ]:
# Please add more stop words here as and when they are encountered

stops_list =['table_end','table_start','president','vice','business','chief','exerience','mhz','usa','senior','www','world',\
'january','february','march',\
'april','may','june','july','august','september','october','november','december','year','mr','north','america','industry',\
'payment','solution','solutions','i','ii','iii','iv','v','vi','vii','viii','company','product','management','customer',\
'item','ability','access','annual','report','broad','related','system','service','uncertain','unauthorized','false',\
'system','uk','revenue','earning','approval','region','example','incident','company','financial','statements','supplementary',\
'data','summary','notes','consolidated','note','notes','due', 'adjusted','review','versus','herein','reference','industrial', \
             'products','overall','result'\
            ]
stops=stops+stops_list

In [2]:
item1_data=pd.read_csv("sp500_item1_sec_filings_0.txt", delimiter="|", parse_dates=['date'])
item1a_data=pd.read_csv("sp500_item1a_sec_filings_0.txt", delimiter="|", parse_dates=['date'])
item7_data=pd.read_csv("sp500_item7_sec_filings_0.txt", delimiter="|", parse_dates=['date'])

In [3]:
item1_data["date"]=pd.to_datetime(item1_data["date"] , utc=True)
item1_data['year']=item1_data["date"].dt.year # Extract year
item1_data["date"]=item1_data["date"].dt.date

item1a_data["date"]=pd.to_datetime(item1a_data["date"] , utc=True)
item1a_data['year']=item1a_data["date"].dt.year
item1a_data["date"]=item1a_data["date"].dt.date

item7_data["date"]=pd.to_datetime(item7_data["date"] , utc=True)
item7_data['year']=item7_data["date"].dt.year
item7_data["date"]=item7_data["date"].dt.date


In [4]:
item1_data.drop_duplicates(subset=['cik','date'], keep='last', inplace=True, ignore_index=True) # Remove duplicates
item1a_data.drop_duplicates(subset=['cik','date'], keep='last', inplace=True, ignore_index=True)
item7_data.drop_duplicates(subset=['cik','date'], keep='last', inplace=True, ignore_index=True)


merged_stage1 = pd.merge(item1_data, item1a_data[['cik','date','company','item1A']], on=["cik","date"], how="inner")
final_merge=pd.merge(merged_stage1, item7_data[['cik','date','company','item7']], on=["cik","date"], how="inner")

In [22]:
final_merge_1= final_merge.dropna(subset=['year', 'item1']) #14718
final_merge_1.drop(['item7','item1A', 'company_y','company_x','link'], axis=1,inplace=True)

final_merge_1A= final_merge.dropna(subset=['year', 'item1A']) #11400
final_merge_1A.drop(['item7','item1','company_y','company_x','link'], axis=1,inplace=True)

final_merge_7= final_merge.dropna(subset=['year', 'item7']) #14667
final_merge_7.drop(['item1','item1A','company_y','company_x','link'], axis=1,inplace=True)

C:\Users\ajaco\AppData\Local\Temp\ipykernel_9784\1201314047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merge_1.drop(['item7','item1A', 'company_y','company_x','link'], axis=1,inplace=True)
C:\Users\ajaco\AppData\Local\Temp\ipykernel_9784\1201314047.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merge_1A.drop(['item7','item1','company_y','company_x','link'], axis=1,inplace=True)
C:\Users\ajaco\AppData\Local\Temp\ipykernel_9784\1201314047.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

In [23]:
final_merge_1A.head(2)

,date,type,cik,year,item1A,company
0,2022-02-09,10-K,66740,2022,Item 1A. Risk Factors Provided below is a cau...,3M CO
1,2021-02-04,10-K,66740,2021,Item 1A. Risk Factors Provided below is a ca...,3M CO


In [25]:
addition = ['ADJ','NOUN', 'PROPN'] #capture tokens that are either Nouns, Proper Nouns or Adjectives


def parse_document(doc):
    
    clean_tweet = re.sub("#[A-Za-z0-9_]+","", doc) # Remove Hash words
    clean_tweet = re.sub("(?:(?:https?|ftp):\\/\\/)?[\\w/\\-?=%.]+\\.[\\w/\\-&?=%.]+", "", clean_tweet) #Remove URL's
    new_tweet = re.sub('(?<!\\w)[!"\\\\$%\\&/#\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~](?!\\w)', \
                       " ", clean_tweet) #Remove special characaters except if between words or numbers
    expanded_words = []
    expanded_words.append(contractions.fix(new_tweet)) #isn't to isnt cases
    expanded_text = ' '.join(expanded_words)
    words = expanded_text.split(' ')
    new_words = ' '.join([x for x in words if (not x.isdigit() and not x.lower() in stops)  ])
    
    doc = nlp(str(new_words))
    words = []
    for i,sent in enumerate(doc.sents):
        
        for word in sent:
            if len(word.text) >= 2 and word.pos_  in addition : # will still skip < 2 letter words 
                # adjectives, noun or proper nouns
                lem = word.lemma_.lower()
                words.append(lem)
        
    return ' '.join(words)

# The below lines can take > 4 hours each depending on processing capacity

In [26]:
final_merge_1['item1_cleaned'] = final_merge_1['item1'].apply(parse_document)
final_merge_1.to_pickle("item1_cleaned.pkl") 

C:\Users\ajaco\AppData\Local\Temp\ipykernel_9784\1403742092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merge_1['item1_cleaned'] = final_merge_1['item1'].apply(parse_document)


In [27]:
final_merge_1A['item1A_cleaned'] = final_merge_1A['item1A'].apply(parse_document)
final_merge_1A.to_pickle("item1A_cleaned.pkl") 

C:\Users\ajaco\AppData\Local\Temp\ipykernel_9784\3653667875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merge_1A['item1A_cleaned'] = final_merge_1A['item1A'].apply(parse_document)


In [28]:
nlp.max_length = 10_000_000
final_merge_7['item7_cleaned'] = final_merge_7['item7'].apply(parse_document)
final_merge_7.to_pickle("item7_cleaned.pkl") 

C:\Users\ajaco\AppData\Local\Temp\ipykernel_9784\3877132198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merge_7['item7_cleaned'] = final_merge_7['item7'].apply(parse_document)
